<a href="https://colab.research.google.com/github/pullz6/Research_inspired_Self-Explaining-Neural-Networks-for-Business-Process-Monitoring/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install graphviz

In [ ]:
#Importing libraries
import pandas as pd
import numpy as np
from graphviz import Digraph
from matplotlib import pyplot as plt

In [ ]:
df = pd.read_csv("Helpdesk.csv")

In [ ]:
df.head()

,concept:name,lifecycle:transition,org:resource,time:timestamp,Activity,Resource,case:concept:name,case:variant,case:variant-index,case:creator
0,Assign seriousness,complete,Value 2,2010-01-13 08:40:25+02:00,Assign seriousness,Value 2,Case3608,Variant 33,33,Fluxicon Disco
1,Take in charge ticket,complete,Value 2,2010-01-29 08:52:27+02:00,Take in charge ticket,Value 2,Case3608,Variant 33,33,Fluxicon Disco
2,Resolve ticket,complete,Value 2,2010-01-29 08:52:34+02:00,Resolve ticket,Value 2,Case3608,Variant 33,33,Fluxicon Disco
3,Closed,complete,Value 5,2010-02-13 08:52:48+02:00,Closed,Value 5,Case3608,Variant 33,33,Fluxicon Disco
4,Closed,complete,Value 5,2010-02-13 08:52:48+02:00,Closed,Value 5,Case3608,Variant 33,33,Fluxicon Disco


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21348 entries, 0 to 21347
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   concept:name          21348 non-null  object
 1   lifecycle:transition  21348 non-null  object
 2   org:resource          21348 non-null  object
 3   time:timestamp        21348 non-null  object
 4   Activity              21348 non-null  object
 5   Resource              21348 non-null  object
 6   case:concept:name     21348 non-null  object
 7   case:variant          21348 non-null  object
 8   case:variant-index    21348 non-null  int64 
 9   case:creator          21348 non-null  object
dtypes: int64(1), object(9)
memory usage: 1.6+ MB


In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
df['time:timestamp'] = df['time:timestamp'].str[:10]

In [ ]:
df['time:timestamp'] = pd.to_datetime(df['time:timestamp'],dayfirst= True, format='mixed')

In [ ]:
df.head()

,concept:name,lifecycle:transition,org:resource,time:timestamp,Activity,Resource,case:concept:name,case:variant,case:variant-index,case:creator
0,Assign seriousness,complete,Value 2,2010-01-13,Assign seriousness,Value 2,Case3608,Variant 33,33,Fluxicon Disco
1,Take in charge ticket,complete,Value 2,2010-01-29,Take in charge ticket,Value 2,Case3608,Variant 33,33,Fluxicon Disco
2,Resolve ticket,complete,Value 2,2010-01-29,Resolve ticket,Value 2,Case3608,Variant 33,33,Fluxicon Disco
3,Closed,complete,Value 5,2010-02-13,Closed,Value 5,Case3608,Variant 33,33,Fluxicon Disco
5,Assign seriousness,complete,Value 2,2010-01-13,Assign seriousness,Value 2,Case2748,Variant 1,1,Fluxicon Disco


In [ ]:
df['concept:name'].unique()

array(['Assign seriousness', 'Take in charge ticket', 'Resolve ticket',
       'Closed', 'Wait', 'Create SW anomaly', 'Insert ticket',
       'Schedule intervention', 'RESOLVED', 'INVALID', 'VERIFIED',
       'Resolve SW anomaly', 'Require upgrade', 'DUPLICATE'], dtype=object)

In [ ]:
#Checking if we can concept:name and activity is the same as well as the org:resource and the resource, so that we can delete this feild.
df['Checker_1'] = (df['concept:name'] == df['Activity'])
df['Checker_2'] = (df['org:resource'] == df['Resource'])

In [ ]:
#Let's see if it all the same, if True.
df['Checker_2'].unique()

array([ True])

In [ ]:
#Lets drop it since it is the same
df.drop(['concept:name','org:resource','Checker_1','Checker_2','case:creator'],axis=1, inplace=True)

In [ ]:
#Lets create a stage number for each activity stage of the help desk, this will help us easily identify where the case is in terms of the process

#This is also to immulate the label encoder
#First create a dataframe with stage numbers and the stage names
stage = pd.DataFrame()
stage['Stage_no'] = [1,2,3,4,5,6,7,8,9,9.1,9.2,10]
stage['Activity'] = ['Insert ticket','Assign seriousness','Take in charge ticket','Wait','Require upgrade','Schedule intervention','Resolve ticket','VERIFIED','RESOLVED','INVALID', 'DUPLICATE','Closed']

In [ ]:
df=pd.merge(df,stage,on='Activity', how='left')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21221 entries, 0 to 21220
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   lifecycle:transition  21221 non-null  object        
 1   time:timestamp        21221 non-null  datetime64[ns]
 2   Activity              21221 non-null  object        
 3   Resource              21221 non-null  object        
 4   case:concept:name     21221 non-null  object        
 5   case:variant          21221 non-null  object        
 6   case:variant-index    21221 non-null  int64         
 7   Stage_no              21142 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 1.3+ MB


In [ ]:
df.head()

,lifecycle:transition,time:timestamp,Activity,Resource,case:concept:name,case:variant,case:variant-index,Stage_no
0,complete,2010-01-13,Assign seriousness,Value 2,Case3608,Variant 33,33,2.0
1,complete,2010-01-29,Take in charge ticket,Value 2,Case3608,Variant 33,33,3.0
2,complete,2010-01-29,Resolve ticket,Value 2,Case3608,Variant 33,33,7.0
3,complete,2010-02-13,Closed,Value 5,Case3608,Variant 33,33,10.0
4,complete,2010-01-13,Assign seriousness,Value 2,Case2748,Variant 1,1,2.0


In [1]:
#Creating the model architecture

In [3]:
#Importing the required libriares since the LSTM model will be done with Pytorch
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt